```
ME72: Maestría en Métodos Cuantitativos para la Gestión y Análisis de Datos
M72109: Analisis de datos no estructurados
Universidad de Buenos Aires - Facultad de Ciencias Economicas (UBA-FCE)
Año: 2020
Profesor: Facundo Santiago
```


# BERT: Bidirectional Encoder Representations from Transformers

In [ ]:
import warnings
warnings.filterwarnings('ignore')

## Preparación del ambiente

Descarguemos algunos fragmentos de código para simplificar el trabajo

In [ ]:
!wget https://raw.githubusercontent.com/santiagxf/M72109/master/NLP/Utils/ClassificationDataset.py --directory-prefix ./Utils/

Descargamos el set de datos

In [ ]:
!wget https://raw.githubusercontent.com/santiagxf/M72109/master/NLP/Datasets/mascorpus/tweets_marketing.csv --directory-prefix ./Datasets/mascorpus/

Instalamos las librerias necesarias

In [ ]:
!pip install transformers

Cargamos el set de datos

In [4]:
import pandas as pd

tweets = pd.read_csv('Datasets/mascorpus/tweets_marketing.csv')

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(tweets['TEXTO'], tweets['SECTOR'], 
                                                    test_size=0.33, 
                                                    stratify=tweets['SECTOR'])

## Introducción a BERT

word2vec es un modelo de tipo "context-free", lo que significa que cada palabra recibe un único vector que la representa. Esto implica que por ejemplo la palabra "banco" recibirá la misma representación en las oraciones *Los domingos no abre el banco* y *Estabamos tan cansados que nos sentamos en un banco*.

BERT, sin embargo, es un modelo contextual, lo que significa que la representación que se genera de una palabra depende del contexto en el que aparece.

### Como funciona

Si recordamos de cuando introducimos word2vec, vimos que las representaciones de las palabras se obtenian al entrenar una red neuronal en una tarea "falsa" que era predecir una palabra dado el contexto en el que aparece. Este contexto lo especificabamos como una ventana de palabras. Los modelos basados en lenguaje, llevan esta tarea un paso más adelante y tratan de predecir la siguiente palabra dada una secuencia de tokens.

En el caso de BERT, está pre-entrenado utilizando 2 tareas distintas:
 - **Masked LM:** BERT está basado en una técnica llamada Masked LM (MLM) la cual, en lugar de intentar predecir la siguiente palabra dada una secuencia de palabras, aleatoriamente enmascara palabras en la oración para luego intentar predecirlas desde el contexto. Para hacerlo utiliza el contexto completo de la oración, tanto hacia adelante como hacía atras (por esto se llama bidireccional). En practica, BERT enmascara aproximadamente el 15% de los tokens en una secuencia.
 - **NSP (Next Sentence Prediction):** Muchas tareas en NLP requieren el entendimiento de las relaciones entre varias oraciones o secuencias. BERT captura estas relaciones al estar entrenado para predecir la siguiente oración en el cuerpo. En realidad BERT utiliza 50% del tiempo efectivamente la siguiente oración para la tarea de NSP y la taguea con el token IsNext, mientras que el otro 50% utiliza una oración aleatoria del texto y la taguea con el token NoNext.

## Explorando un modelo pre-entrenado con BERT

Una de las formas más sencillas de trabajar con el modelo BERT es utilizando la libreria transformers de Hugging Face [https://huggingface.co/] la cual ofrece una forma muy conveniente de acceder a modelos de NLP en diferentes lenguajes e incluso entrenados para tareas especificas.

Podemos instalar esta libreria desde pip de la siguiente forma. Este paso ya lo realizamos en la sección de instalación de este notebook

```
pip install transformers
```

### BETO: BERT en español

Al igual que con word2vec, entrenar un modelo de lenguaje requiere de una gran cantidad de datos sumado a un poder de computo interesante (cuando BERT fué publicado en 2018, tomó 4 días entrenar el modelo usando 16 TPUs. Si se hubiera entrenado en 8 GPUs hubiera tomado entre 40–70 días).Por este motivo, utilizaremos un modelo pre-entrenado para un cuerpo de texto en español. Este modelo, BETO, fué entrenado sobre un gran corpora de texto. Pueden encontrar más información sobre el autor de este modelo en: https://github.com/dccuchile/beto 

### Tokenizers 

BERT utiliza su propio tokenizer que está basado en WordPiece. Este tokenizer tiene un vocabulario de 30.000 tokens donde cada secuencia comienza con un token especial [CLS]. Exploremos como funciona este tokenizer

In [ ]:
import transformers

tokenizer = transformers.BertTokenizerFast.from_pretrained('dccuchile/bert-base-spanish-wwm-uncased', do_lower_case=True)

*Noten que el tokenizer depende del modelo que estamos utilizando*

Exploremos los tokens que genera:

In [7]:
text = tweets['TEXTO'][5]

In [8]:
tokens = tokenizer.encode(text)

¿Notan algo raro en los tokens generados?

In [ ]:
tokenizer.decode(tokens)

¿Siguen notando algo raro?

In [ ]:
[tokenizer.convert_ids_to_tokens(idx) for idx in tokens]

### Cargando nuestro modelo de BERT para español

Para cargar nuestro modelo, utilizaremos la libreria transformers nuevamente, utilizando el método from_pretrained. Este método descargará automaticamente el modelo desde el directorio de modelos de HuggingFace. Pueden ver el listado de modelos que están disponibles en este directorio en [https://huggingface.co/models]

In [ ]:
model = transformers.BertForMaskedLM.from_pretrained('dccuchile/bert-base-spanish-wwm-uncased')

Veamos como se comporta nuestro modelo en la tarea de predecir una palabra de un texto:

In [13]:
text = "[CLS] Cuando [MASK] contaron lo que sucedia nos quedamos helados. [SEP]"
tokens = tokenizer.tokenize(text)

Noten que el token [MASK] es la palabra que estamos intentando predecir

Necesitamos saber cual de todos los tokens que generamos es exactamente el que enmascaramos:

In [14]:
masked_indxs = [idx for idx in range(0, len(tokens)) if tokens[idx] == '[MASK]']

Corremos nuestro modelo:

In [16]:
import torch

indexed_tokens = tokenizer.convert_tokens_to_ids(tokens)
tokens_tensor = torch.tensor([indexed_tokens])

predictions = model(tokens_tensor)[0]

Verificamos cuales son las palabras más probables:

In [ ]:
for i,midx in enumerate(masked_indxs):
    idxs = torch.argsort(predictions[0,midx], descending=True)
    predicted_token = tokenizer.convert_ids_to_tokens(idxs[:5])
    print('Las 5 palabras más probables para la mascara',i,'son:',predicted_token)